<a href="https://colab.research.google.com/github/olegsamara-63/for_rstelecom_testers/blob/main/My_framework_for_RT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title

# %% [markdown]
# # Фреймворк для автоматизированного тестирования пользовательских эндпоинтов
#
# ## Установка необходимых библиотек

# %%
!pip install requests pytest faker pytest-html

# %%
import requests
import pytest
import json
import random
from faker import Faker
from typing import Dict, Any
import datetime
import sys

# %% [markdown]
# ## Базовый класс для тестов

# %%
class UserAPITestBase:
    """Базовый класс для тестирования API пользователей"""

    BASE_URL = "https://petstore.swagger.io/v2"

    def __init__(self):
        self.fake = Faker()
        self.session = requests.Session()
        self.test_results = []

    def generate_user_data(self) -> Dict[str, Any]:
        """Генерация случайных тестовых данных пользователя"""
        return {
            "id": random.randint(1000, 9999),
            "username": self.fake.user_name(),
            "firstName": self.fake.first_name(),
            "lastName": self.fake.last_name(),
            "email": self.fake.email(),
            "password": self.fake.password(length=12),
            "phone": self.fake.phone_number(),
            "userStatus": random.randint(0, 1)
        }

    def create_user(self, user_data: Dict[str, Any]) -> requests.Response:
        """Создание пользователя"""
        url = f"{self.BASE_URL}/user"
        headers = {'Content-Type': 'application/json'}
        return self.session.post(url, data=json.dumps(user_data), headers=headers)

    def login_user(self, username: str, password: str) -> requests.Response:
        """Вход пользователя"""
        url = f"{self.BASE_URL}/user/login?username={username}&password={password}"
        return self.session.get(url)

    def logout_user(self) -> requests.Response:
        """Выход пользователя"""
        url = f"{self.BASE_URL}/user/logout"
        return self.session.get(url)

    def get_user(self, username: str) -> requests.Response:
        """Получение информации о пользователе"""
        url = f"{self.BASE_URL}/user/{username}"
        return self.session.get(url)

    def update_user(self, username: str, user_data: Dict[str, Any]) -> requests.Response:
        """Обновление пользователя"""
        url = f"{self.BASE_URL}/user/{username}"
        headers = {'Content-Type': 'application/json'}
        return self.session.put(url, data=json.dumps(user_data), headers=headers)

    def delete_user(self, username: str) -> requests.Response:
        """Удаление пользователя"""
        url = f"{self.BASE_URL}/user/{username}"
        return self.session.delete(url)

    def add_test_result(self, test_name: str, status: str, response: requests.Response,
                       expected_status: int = None, notes: str = ""):
        """Добавление результата теста"""
        result = {
            "test_name": test_name,
            "status": status,
            "timestamp": datetime.datetime.now().isoformat(),
            "response_status": response.status_code,
            "expected_status": expected_status,
            "response_body": response.json() if response.content else {},
            "notes": notes
        }
        self.test_results.append(result)

    def generate_report(self):
        """Генерация отчета о тестировании"""
        total_tests = len(self.test_results)
        passed_tests = sum(1 for result in self.test_results if result["status"] == "PASSED")
        failed_tests = total_tests - passed_tests

        report = {
            "report_date": datetime.datetime.now().isoformat(),
            "total_tests": total_tests,
            "passed_tests": passed_tests,
            "failed_tests": failed_tests,
            "success_rate": (passed_tests / total_tests * 100) if total_tests > 0 else 0,
            "test_results": self.test_results
        }

        # Сохранение отчета в файл
        with open("test_report.json", "w") as f:
            json.dump(report, f, indent=2)

        # Вывод отчета в консоль
        print("=" * 60)
        print("ОТЧЕТ О ТЕСТИРОВАНИИ API ПОЛЬЗОВАТЕЛЕЙ")
        print("=" * 60)
        print(f"Дата отчета: {datetime.datetime.now()}")
        print(f"Всего тестов: {total_tests}")
        print(f"Пройдено: {passed_tests}")
        print(f"Не пройдено: {failed_tests}")
        print(f"Успешность: {report['success_rate']:.2f}%")
        print("=" * 60)

        for result in self.test_results:
            status_icon = "✅" if result["status"] == "PASSED" else "❌"
            print(f"{status_icon} {result['test_name']}: {result['status']}")
            if result["status"] == "FAILED":
                print(f"   Ожидалось: {result['expected_status']}, Получено: {result['response_status']}")
                print(f"   Примечание: {result['notes']}")

        return report

# %% [markdown]
# ## Тесты для создания пользователя (POST /user)

# %%
class TestUserCreation(UserAPITestBase):
    """Тесты для создания пользователей"""

    def test_create_user_success(self):
        """Тест успешного создания пользователя"""
        user_data = self.generate_user_data()
        response = self.create_user(user_data)

        if response.status_code == 200:
            self.add_test_result("test_create_user_success", "PASSED", response, 200)
            return user_data
        else:
            self.add_test_result("test_create_user_success", "FAILED", response, 200,
                               "Не удалось создать пользователя")
            return None

    def test_create_user_invalid_data(self):
        """Тест создания пользователя с невалидными данными"""
        invalid_data = {"invalid": "data"}
        url = f"{self.BASE_URL}/user"
        response = self.session.post(url, data=json.dumps(invalid_data),
                                   headers={'Content-Type': 'application/json'})

        # PetStore может возвращать 200 даже на невалидные данные, поэтому проверяем наличие ошибки в ответе
        if response.status_code != 200 or "error" in str(response.json()).lower():
            self.add_test_result("test_create_user_invalid_data", "PASSED", response,
                               "any error", "Ожидалась ошибка при невалидных данных")
        else:
            self.add_test_result("test_create_user_invalid_data", "FAILED", response,
                               "error expected", "Не возвращена ошибка на невалидные данные")

# %% [markdown]
# ## Тесты для входа и выхода пользователя

# %%
class TestUserLoginLogout(UserAPITestBase):
    """Тесты для входа и выхода пользователя"""

    def test_login_logout_success(self):
        """Тест успешного входа и выхода пользователя"""
        # Сначала создаем пользователя
        user_data = self.generate_user_data()
        create_response = self.create_user(user_data)

        if create_response.status_code != 200:
            self.add_test_result("test_login_logout_success", "FAILED", create_response, 200,
                               "Не удалось создать пользователя для теста входа")
            return

        # Тестируем вход
        login_response = self.login_user(user_data["username"], user_data["password"])

        if login_response.status_code == 200 and "logged in" in login_response.text.lower():
            self.add_test_result("test_login_success", "PASSED", login_response, 200)
        else:
            self.add_test_result("test_login_success", "FAILED", login_response, 200,
                               "Не удалось войти пользователя")
            return

        # Тестируем выход
        logout_response = self.logout_user()

        if logout_response.status_code == 200 and "ok" in logout_response.text.lower():
            self.add_test_result("test_logout_success", "PASSED", logout_response, 200)
        else:
            self.add_test_result("test_logout_success", "FAILED", logout_response, 200,
                               "Не удалось выйти пользователя")

    def test_login_invalid_credentials(self):
        """Тест входа с неверными учетными данными"""
        response = self.login_user("nonexistent_user", "wrong_password")

        # PetStore может возвращать 200 даже на неверные credentials, проверяем сообщение
        if response.status_code == 200 and "invalid" in response.text.lower():
            self.add_test_result("test_login_invalid_credentials", "PASSED", response,
                               "any error", "Ожидалась ошибка при неверных учетных данных")
        else:
            self.add_test_result("test_login_invalid_credentials", "FAILED", response,
                               "error expected", "Не возвращена ошибка при неверных учетных данных")

# %% [markdown]
# ## Тесты для обновления и удаления пользователя

# %%
class TestUserUpdateDelete(UserAPITestBase):
    """Тесты для обновления и удаления пользователя"""

    def test_update_user_success(self):
        """Тест успешного обновления пользователя"""
        # Создаем пользователя
        user_data = self.generate_user_data()
        create_response = self.create_user(user_data)

        if create_response.status_code != 200:
            self.add_test_result("test_update_user_success", "FAILED", create_response, 200,
                               "Не удалось создать пользователя для теста обновления")
            return

        # Обновляем данные пользователя
        updated_data = user_data.copy()
        updated_data["firstName"] = "UpdatedFirstName"
        updated_data["email"] = "updated@example.com"

        update_response = self.update_user(user_data["username"], updated_data)

        if update_response.status_code == 200:
            self.add_test_result("test_update_user_success", "PASSED", update_response, 200)

            # Проверяем, что данные действительно обновились
            get_response = self.get_user(user_data["username"])
            if (get_response.status_code == 200 and
                get_response.json()["firstName"] == "UpdatedFirstName"):
                self.add_test_result("test_verify_update", "PASSED", get_response, 200)
            else:
                self.add_test_result("test_verify_update", "FAILED", get_response, 200,
                                   "Данные пользователя не обновились")
        else:
            self.add_test_result("test_update_user_success", "FAILED", update_response, 200,
                               "Не удалось обновить пользователя")

    def test_delete_user_success(self):
        """Тест успешного удаления пользователя"""
        # Создаем пользователя
        user_data = self.generate_user_data()
        create_response = self.create_user(user_data)

        if create_response.status_code != 200:
            self.add_test_result("test_delete_user_success", "FAILED", create_response, 200,
                               "Не удалось создать пользователя для теста удаления")
            return

        # Удаляем пользователя
        delete_response = self.delete_user(user_data["username"])

        if delete_response.status_code == 200:
            self.add_test_result("test_delete_user_success", "PASSED", delete_response, 200)

            # Проверяем, что пользователь действительно удален
            get_response = self.get_user(user_data["username"])
            if get_response.status_code == 404:
                self.add_test_result("test_verify_deletion", "PASSED", get_response, 404)
            else:
                self.add_test_result("test_verify_deletion", "FAILED", get_response, 404,
                                   "Пользователь не был удален")
        else:
            self.add_test_result("test_delete_user_success", "FAILED", delete_response, 200,
                               "Не удалось удалить пользователя")

# %% [markdown]
# ## Запуск всех тестов и генерация отчета

# %%
def run_all_tests():
    """Запуск всех тестов и генерация отчета"""

    print("Запуск тестов API пользователей...")
    print("=" * 50)

    # Создаем экземпляры тестовых классов
    creation_tests = TestUserCreation()
    login_tests = TestUserLoginLogout()
    update_tests = TestUserUpdateDelete()

    # Запускаем тесты создания
    print("Тестирование создания пользователей...")
    user_data = creation_tests.test_create_user_success()
    creation_tests.test_create_user_invalid_data()

    # Запускаем тесты входа/выхода
    print("Тестирование входа/выхода пользователей...")
    login_tests.test_login_logout_success()
    login_tests.test_login_invalid_credentials()

    # Запускаем тесты обновления/удаления
    print("Тестирование обновления и удаления пользователей...")
    update_tests.test_update_user_success()
    update_tests.test_delete_user_success()

    # Собираем все результаты
    all_results = (creation_tests.test_results +
                  login_tests.test_results +
                  update_tests.test_results)

    # Создаем финальный отчет
    final_report = UserAPITestBase()
    final_report.test_results = all_results
    report = final_report.generate_report()

    print("\nТестирование завершено!")
    return report

# %%
# Запускаем все тесты
if __name__ == "__main__":
    test_report = run_all_tests()

    # Дополнительная информация для отладки
    print("\n" + "=" * 60)
    print("ДЕТАЛИ ОТЧЕТА СОХРАНЕНЫ В ФАЙЛЕ: test_report.json")
    print("=" * 60)

# %% [markdown]
# ## Дополнительные утилиты для тестирования

# %%
def run_specific_test(test_type: str):
    """Запуск конкретного типа тестов"""

    if test_type.lower() == "create":
        tests = TestUserCreation()
        tests.test_create_user_success()
        tests.test_create_user_invalid_data()
        tests.generate_report()

    elif test_type.lower() == "login":
        tests = TestUserLoginLogout()
        tests.test_login_logout_success()
        tests.test_login_invalid_credentials()
        tests.generate_report()

    elif test_type.lower() == "update":
        tests = TestUserUpdateDelete()
        tests.test_update_user_success()
        tests.test_delete_user_success()
        tests.generate_report()

    else:
        print("Доступные типы тестов: create, login, update")

# %%
def test_single_user_flow():
    """Тест полного цикла работы с пользователем"""
    print("Тестирование полного цикла пользователя...")

    base = UserAPITestBase()
    user_data = base.generate_user_data()

    # 1. Создание пользователя
    print("1. Создание пользователя...")
    response = base.create_user(user_data)
    base.add_test_result("full_flow_create",
                       "PASSED" if response.status_code == 200 else "FAILED",
                       response, 200)

    # 2. Вход пользователя
    print("2. Вход пользователя...")
    response = base.login_user(user_data["username"], user_data["password"])
    base.add_test_result("full_flow_login",
                       "PASSED" if response.status_code == 200 else "FAILED",
                       response, 200)

    # 3. Получение информации о пользователе
    print("3. Получение информации о пользователе...")
    response = base.get_user(user_data["username"])
    base.add_test_result("full_flow_get",
                       "PASSED" if response.status_code == 200 else "FAILED",
                       response, 200)

    # 4. Обновление пользователя
    print("4. Обновление пользователя...")
    updated_data = user_data.copy()
    updated_data["firstName"] = "UpdatedName"
    response = base.update_user(user_data["username"], updated_data)
    base.add_test_result("full_flow_update",
                       "PASSED" if response.status_code == 200 else "FAILED",
                       response, 200)

    # 5. Выход пользователя
    print("5. Выход пользователя...")
    response = base.logout_user()
    base.add_test_result("full_flow_logout",
                       "PASSED" if response.status_code == 200 else "FAILED",
                       response, 200)

    # 6. Удаление пользователя
    print("6. Удаление пользователя...")
    response = base.delete_user(user_data["username"])
    base.add_test_result("full_flow_delete",
                       "PASSED" if response.status_code == 200 else "FAILED",
                       response, 200)

    base.generate_report()

# %% [markdown]
# ## Примеры использования

# %%
# Пример 1: Запуск всех тестов
# run_all_tests()

# %%
# Пример 2: Запуск конкретного типа тестов
# run_specific_test("create")

# %%
# Пример 3: Тестирование полного цикла одного пользователя
# test_single_user_flow()

# %%
# Пример 4: Генерация тестовых данных
base = UserAPITestBase()
test_user = base.generate_user_data()
print("Пример тестовых данных пользователя:")
print(json.dumps(test_user, indent=2))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.9 MB/s eta 0:00:00
Запуск тестов API пользователей...
Тестирование создания пользователей...
Тестирование входа/выхода пользователей...
Тестирование обновления и удаления пользователей...
ОТЧЕТ О ТЕСТИРОВАНИИ API ПОЛЬЗОВАТЕЛЕЙ
Дата отчета: 2025-08-28 20:31:02.327345
Всего тестов: 8
Пройдено: 4
Не пройдено: 4
Успешность: 50.00%
✅ test_create_user_success: PASSED
❌ test_create_user_invalid_data: FAILED
   Ожидалось: error expected, Получено: 200
   Примечание: Не возвращена ошибка на невалидные данные
✅ test_login_success: PASSED
✅ test_logout_success: PASSED
❌ test_login_invalid_credentials: FAILED
   Ожидалось: error expected, Получено: 200
   Примечание: Не возвращена ошибка при неверных учетных данных
✅ test_update_user_success: PASSED
❌ test_verify_update: FAILED
   Ожидалось: 200, Получено: 200
   Примечание: Данные пользователя не обновились
❌ test_delete_user_success: FAILED
   Ожидалось: 200, Получено: 404
   Примечание: 